In [ ]:
# Imports
from IOHexperimenter import IOH_function, IOH_logger
from IOH_function_wrapper import IOH_function_logger
import input_reader
import parameters
import datetime
from shutil import make_archive
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a dictionary that contains algorithms and splitpoints / targets
data = input_reader.create_dict()

In [ ]:
class Switch:
    """ Class to implement the switch between algorithms.

    Parameters:
    ---------------
    func : object, callable
        The function to be optimized.

    budget : int
        The budget that is available for function evaluations.

    data : dict
        A dictionary with tau as key, algorithm as value


    Attibutes:
    ---------------
    None

    """

    def __init__(self, func, budget, data, seed=None):
        np.random.seed(seed)
        self.func = func
        self.budget = budget * func.number_of_variables
        self.data = data

    def __call__(self):
        """ Runs the switch routine. 
        
        Returns:
        -----------
        best_so_far_variables : array
                The best found solution.

        best_so_far_fvaluet: float
               The fitness value for the best found solution.
        """

        # Initialize parameters
        params = parameters.init_params(self.budget)

        # Iterate through dictionary
        for tau, alg in data.items():
            algorithm = alg(self.func)
            # Set algorithm parameters based on parameters object

            algorithm.set_params(params)

            # Define stopping criteria
            func = self.func
            budget = self.budget

            def stopping_criteria():
                return (func.best_so_far_precision) <= tau or (
                    func.evaluations >= budget)

            algorithm.set_stopping_criteria(stopping_criteria)

            # Run algorithm and extract parameters
            algorithm.run()
            params = algorithm.get_params(params)
            
            params.internal_dict['switch_prec'] = self.func.best_so_far_precision

            if self.func.evaluations >= budget:
                break

        # Return best point and fitness value found by last run algorithm
        return (algorithm.func.best_so_far_variables, algorithm.func.best_so_far_precision, params) 

In [ ]:
# Experiment routine
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/report-data/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
dimensions = [2]
instances = [1, 2, 3, 4, 5]
func_ids = [24]
exp_budget = 10000
runs = 5
switch_combination = "CMAES_BFGS"
settings = "_default"
experiment_name = path + switch_combination + "_experiment" + experiment_ID + "_" + settings + "_f" + str(func_ids[0]) + "d" +  str(dimensions[0])

for func_id in func_ids:
    for dim in dimensions:
        for inst in instances:
            f = IOH_function_logger(func_id, dim, inst, suite='BBOB')
            log = IOH_logger(experiment_name, "run",
                         switch_combination + settings, "switch-routine")
            f.add_logger(log)
            for run in range(0, runs):
                if run > 0:
                    f.reset()
                    f.x_hist = []
                    f.f_hist = []
                switch = Switch(f, exp_budget, data)
                x_opt, prec, params = switch()
                print(f' run {run} dim{dim} f{func_id} inst{inst} complete, precision: {prec}')

            f.clear_logger()

make_archive(experiment_name, "zip", experiment_name)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_hist_pso = params.internal_dict['pso_x_hist']
x_opt_pso = params.internal_dict['pso_x_opt']
gens_pso = params.internal_dict['pso_gen_counter']

x1_coords_pso = []
x2_coords_pso = []
for i in range(0, len(x_hist_pso)):
    x1_coords_pso.append(x_hist_pso[i][0])
    x2_coords_pso.append(x_hist_pso[i][1])


"""x_hist_mlsl = params.internal_dict['mlsl_x_hist']
x_opt_mlsl = params.internal_dict['mlsl_x_opt']
gens_mlsl = params.internal_dict['mlsl_gen_counter']

x1_coords_mlsl = []
x2_coords_mlsl = []
for i in range(0, len(x_hist_mlsl)):
    x1_coords_mlsl.append(x_hist_mlsl[i][0])
    x2_coords_mlsl.append(x_hist_mlsl[i][1])"""

"""x_hist_de = params.internal_dict['de_x_hist']
x_opt_de = params.internal_dict['de_x_opt']
gens_de = params.internal_dict['de_gen_counter']

x1_coords_de = []
x2_coords_de = []
for i in range(0, len(x_hist_de)):
    x1_coords_de.append(x_hist_de[i][0])
    x2_coords_de.append(x_hist_de[i][1])"""

"""x_hist_cmaes = params.internal_dict['cmaes_x_hist']
x_opt_cmaes = params.internal_dict['cmaes_x_opt']
gens_cmaes = params.internal_dict['cmaes_gen_counter']

x1_coords_cmaes = []
x2_coords_cmaes = []
for i in range(0, len(x_hist_cmaes)):
#for i in range(0, 10000):
    x1_coords_cmaes.append(x_hist_cmaes[i][0])
    x2_coords_cmaes.append(x_hist_cmaes[i][1])"""

"""x_hist_bfgs = params.internal_dict['bfgs_x_hist']
x_opt_bfgs = params.internal_dict['bfgs_x_opt']
gens_bfgs = params.internal_dict['bfgs_gen_counter']

x1_coords_bfgs = []
x2_coords_bfgs = []
for i in range(0, len(x_hist_bfgs)):
    x1_coords_bfgs.append(x_hist_bfgs[i][0])
    x2_coords_bfgs.append(x_hist_bfgs[i][1])"""

In [ ]:
# Contour plot
#array([3.656 , 2.5496])
x_lb = -5
x_ub = 5
y_lb = -5
y_ub = 5
"""x_lb = 3.653
x_ub = 3.659
y_lb = 2.546
y_ub = 2.552"""
zoom_range = (x_ub - x_lb) / 10
sample_size = 100
x_step = ((x_ub +zoom_range - (x_lb-zoom_range)) / sample_size)
y_step = ((y_ub+zoom_range - (y_lb-zoom_range)) / sample_size)
x_coords = np.arange(x_lb-zoom_range, x_ub+zoom_range, x_step)
y_coords = np.arange(y_lb-zoom_range, y_ub+zoom_range, y_step)
X, Y = np.meshgrid(x_coords[0:sample_size], y_coords[0:sample_size])

In [ ]:
Zlog = np.zeros((sample_size, sample_size))
for i in range(0, sample_size):
    for j in range(0, sample_size):
        Zlog[i][j] = np.log10(f((X[i][j], Y[i][j])) - f.get_target())
print(f'min: {np.amin(Zlog)} max: {np.amax(Zlog)}')

In [ ]:
#### import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(10,10))  # Create a figure containing a single axes.
levels = np.arange(-0.5, 5, 0.1)
cs = ax.contourf(X, Y, Zlog, cmap = 'viridis', levels=levels)

"""## MLSL
cm_mlsl = "YlOrBr"
c_mlsl = "yellow"

#sc_mlsl = plt.scatter(x1_coords_mlsl, x2_coords_mlsl, c = c_mlsl, marker = "^")
#ax.plot(x1_coords_bestpoints, x2_coords_bestpoints, 'ro', label = "best-40(MLSL)", c = 'sienna', marker = "^")
#ax.plot(x1_coords_xstar, x2_coords_xstar, 'ro', label = "x_star(MLSL)", c = 'limegreen', marker = "^")
#ax.plot(-1.250003,  1.24999844, 'ro', label = "Optimum", color = "magenta", marker = "*")
sc_mlsl = plt.scatter(x1_coords_mlsl, x2_coords_mlsl, c = gens_mlsl,  cmap = cm_mlsl, marker = "^", alpha=1)
ax.plot(x_opt_mlsl[0], x_opt_mlsl[1], 'ro', label = "x_opt(MLSL)", color = 'cyan', marker = "^", alpha=1)
plt.colorbar(sc_mlsl, label= 'MLSL generation number')"""

## PSO
cm_pso = "Blues"
c_pso = "blue"
sc_pso = plt.scatter(x1_coords_pso[(len(x1_coords_pso)-40):len(x1_coords_pso)], x2_coords_pso[(len(x2_coords_pso)-40):len(x2_coords_pso)], label= 'PSO', c = c_pso, marker = "v", edgecolor="lightblue")
#sc_pso = plt.scatter(x1_coords_pso, x2_coords_pso, c = gens_pso,  cmap = cm_pso, marker = "v", alpha=0.5)
ax.plot(x_opt_pso[0], x_opt_pso[1], 'ro', label = "x_opt(PSO)", color = 'cyan', marker = "v", alpha=1)
#plt.colorbar(sc_pso, label='PSO generation number')

"""
## DE
cm_de = "Oranges"
c_de = "orange"
#sc_de = plt.scatter(x1_coords_de, x2_coords_de, label= 'DE', c = c_de, marker = "s")
sc_de = plt.scatter(x1_coords_de, x2_coords_de, c = gens_de,  cmap = cm_de, marker = "s", alpha=0.8)
ax.plot(x_opt_de[0], x_opt_de[1], 'ro', label = "x_opt(DE)", color = 'magenta', marker = "s", alpha=0.2)
plt.colorbar(sc_de, label='DE generation number')"""

"""## CMAES
cm_cmaes = "Purples"
c_cmaes = "purple"
#sc_cmaes = plt.scatter(x1_coords_cmaes, x2_coords_cmaes, label= 'CMAES', c = c_cmaes, marker = "D", alpha=0.1)
sc_cmaes = plt.scatter(x1_coords_cmaes, x2_coords_cmaes, c = gens_cmaes,  cmap = cm_cmaes, marker = "D", alpha=0.5)
ax.plot(x_opt_cmaes[0], x_opt_cmaes[1], 'ro', label = "x_opt(CMA-ES)", color = 'magenta', marker = "D", alpha=0.8)
plt.colorbar(sc_cmaes, label='CMA-ES generation number')"""

"""## BFGS
cm_bfgs = "Reds"
c_bfgs = "red"
ax.plot(x_opt_bfgs[0], x_opt_bfgs[1], 'ro', label = "x_opt(BFGS)", color = 'magenta', marker = "o", alpha=1)
#sc_bfgs = plt.scatter(x1_coords_bfgs, x2_coords_bfgs, label= 'BFGS', c = c_bfgs, marker = "o", alpha=1)
sc_bfgs = plt.scatter(x1_coords_bfgs, x2_coords_bfgs, c = gens_bfgs,  cmap = cm_bfgs, marker = "o", alpha=1)
plt.colorbar(sc_bfgs, label='BFGS generation number')
"""

#ax.plot(-1.250000,  1.250000, 'ro', label="optimum", color = "red", marker="*", markersize=12)
ax.set_xlabel('x1')  # Add an x-label to the axes.
ax.set_ylabel('x2')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
ax.set_xlim(left=x_lb, right=x_ub)
ax.set_ylim(bottom=y_lb, top=y_ub)
#ax.axis('equal')

plt.savefig(switch_combination + settings + 'with_edgecolor_tau1e-8')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
evals = params.internal_dict['evalcount']
stepsizes = params.internal_dict['stepsizes']
#splitpoint_evals = params.internal_dict['evals_splitpoint']
fig, ax = plt.subplots(figsize=(10, 10))  # Create a figure containing a single axes.
ax.plot(evals, stepsizes, label = 'sigma(CMAES)', color='purple')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('step size')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
#plt.vlines(splitpoint_evals, 0, max(stepsizes)+0.2, colors='red', linestyles='dotted', label='split point')
#ax.set_xlim(left=0, right=800, emit=True, auto=False)
#ax.set_ylim(bottom=0, top=0.01)
plt.savefig(switch_combination + settings + '_stepsize')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
evals = params.internal_dict['evalcount']
matrix_norm = params.internal_dict['matrix_norm']
splitpoint_evals = params.internal_dict['evals_splitpoint']
fig, ax = plt.subplots(figsize=(6,12))  # Create a figure containing a single axes.
ax.plot(evals, matrix_norm, 'ro', label = 'matrix norm(switch)')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('matrix norm')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
#plt.vlines(splitpoint_evals, 0, max(matrix_norm)+1, colors='red', linestyles='dotted', label='split point')
#ax.set_xlim(left=0, right=800, emit=True, auto=False)
ax.set_ylim(bottom=0, top=3)
#plt.savefig(switch_combination + settings + '_matrixnorm_zoom1')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
evals = params.internal_dict['evalcount']
vel = np.asarray(params.internal_dict['vel_overtime'])
fig, ax = plt.subplots(figsize=(12,12))  # Create a figure containing a single axes.
for i in range(0, 40):
    ax.plot(evals, vel[:,i], '-o', label = ('particle_' + str(i) + '_vel'), alpha=0.5)
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('velocity')  # Add a y-label to the axes.
#ax.legend()  # Add a legend.
#plt.vlines(splitpoint_evals, 0, max(matrix_norm)+1, colors='red', linestyles='dotted', label='split point')
#ax.set_xlim(left=0, right=800, emit=True, auto=False)
#ax.set_ylim(bottom=0, top=1)
plt.savefig(switch_combination + settings + '_particle_velocities_overview')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(evals, stepsizes, label = 'stepsizes(switch)')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('step size')  # Add a y-label to the axes.
#ax.set_xlim(left=, right=5000, emit=True, auto=False)
ax.legend()  # Add a legend.
plt.vlines(1659, 0, 5, colors='red', linestyles='dotted', label='split point')
#plt.savefig('sigma-change-switch-bfgs-cmaes-zoom')

In [ ]:
x1_coords_xstar = []
x2_coords_xstar = []
for i in range(0, len(x_star)):
    x1_coords_xstar.append(x_star[i][0])
    x2_coords_xstar.append(x_star[i][1])

In [ ]:
x1_coords_xstar

In [ ]:
f_hist_mlsl = params.internal_dict['mlsl_f_hist']
m = np.hstack((np.asarray(x_hist_mlsl), np.expand_dims(np.asarray(f_hist_mlsl), axis=1)))
sorted_m = m[np.argsort(m[:, f.number_of_variables])]
best_points = sorted_m[0:40, 0:f.number_of_variables]
x1_coords_bestpoints = []
x2_coords_bestpoints = []
for i in range(0, len(best_points)):
    x1_coords_bestpoints.append(best_points[i][0])
    x2_coords_bestpoints.append(best_points[i][1])